# Demos: Lecture 15

## Demo 1: Variational Quantum Classifier

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from lecture14_helpers import *

import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

In [ ]:
n_samples = 100

X, y = make_blobs(
    n_samples=n_samples, 
    centers=[[0.1, 0.2], [0.25, 0.4]], 
    cluster_std=0.05, 
    n_features=2,
    random_state=6
)

In [ ]:
test_X, test_y = make_blobs(
    n_samples=50, 
    centers=[[0.1, 0.2], [0.25, 0.4]], 
    cluster_std=0.05, 
    n_features=2,
    random_state=50
)

In [ ]:
plot_data(X, y)

In [ ]:
X = np.array(X, requires_grad=False)
y = np.array(y, requires_grad=False)
y[y == 0] = -1
test_y[test_y == 0] = -1

### VQC model 1: amplitude embedding

<img src="fig/model1.png" width="500px">

In [ ]:
dev = qml.device('default.qubit', wires=1)

@qml.qnode(dev)
def model_1(point, weights):
    qml.AmplitudeEmbedding(point, wires=0, normalize=True)
    
    qml.Rot(*weights, wires=0)
    
    return qml.expval(qml.PauliZ(0))

In [ ]:
weights = np.random.normal(size=3)

In [ ]:
weights

In [ ]:
def loss(weights):
    loss_sum = 0.0
    
    for idx in range(n_samples):
        point = X[idx]
        true_expval = y[idx]
        
        estimated_expval = model_1(point, weights)
        loss_sum += (estimated_expval - true_expval) ** 2
        
    return loss_sum / n_samples

In [ ]:
our_preds = make_predictions(X, model_1, weights)

In [ ]:
plot_data(X, our_preds)

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.1)

n_its = 100

loss_track = []

for it in range(n_its):
    weights, _loss = opt.step_and_cost(loss, weights)
    if it % 5 == 0:
        our_preds = make_predictions(X, model_1, weights)
        print(f"Loss at iteration {it} = {_loss}  Accuracy = {compute_accuracy(our_preds, y)}")
    loss_track.append(_loss)

In [ ]:
our_preds = make_predictions(X, model_1, weights)

In [ ]:
plot_data(X, our_preds)

In [ ]:
compute_accuracy(our_preds, y)

In [ ]:
test_preds = make_predictions(test_X, model_1, weights)

In [ ]:
compute_accuracy(test_preds, test_y)

### VQC model 2: angle embedding

<img src="fig/model2.png" width="500px">

### VQC model 3: multi-layer angle embedding

<img src="fig/model3.png" width="500px">

## Demo 2: Hamiltonians

## Demo 3: Variational Quantum Eigensolver

<img src="fig/VQE-generic-ansatz.png">

<img src="fig/VQE-problem-specific-ansatz.png">